In [1]:
import scipy as sp
import scipy.interpolate
import numpy as np
import glob
from tabulate import tabulate #to export in table format
import scipy.interpolate as interp

In [2]:
sn_names=glob.glob("*.dat")
print(len(sn_names))

109


In [3]:
file = open("factors.txt")
lines = file.readlines()
file.close()

In [4]:
max_names = []
max_values = []

In [5]:
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    max_names.append(co[0])
    max_values.append(co[1])

In [6]:
new_names = []

In [7]:
x = np.linspace(-10,50,61)
y = np.linspace(3000,9000,601)

In [8]:
X = []

In [9]:
for k in range(0,len(sn_names)):
    
    if sn_names[k][:-4] in max_names:
        
        idx = max_names.index(sn_names[k][:-4])
        #print(float(max_values[idx]))
    
        file = open(sn_names[k])
        lines = file.readlines()
        file.close()

        xtemp = []

        for line in lines:

            if line.startswith('#'): continue
            co=line.rstrip().replace('INDEF','Nan').split()

            xtemp.append(co[0])

        xtemp = np.array(xtemp, dtype=float)
        
        xtemp = xtemp*float(max_values[idx])
        

        #xtemp = xtemp/max(xtemp)
        
        

        if max(xtemp) < 10000000000:
            
            new_names.append(sn_names[k])
            z = np.reshape(xtemp, (601, 61))

            zz = []
            xx = []
            yy = []

            for i in range(0,len(y)):

                for j in range(0,len(x)):

                    zz.append(z[i][j])
                    xx.append(x[j])
                    yy.append(y[i])

            X.append(np.array(zz))


In [10]:
import os.path

In [11]:
x_grid = np.linspace(-10,50,61)
y_grid = np.linspace(3000,9000,601)
B1, B2 = np.meshgrid(x_grid, y_grid, indexing='xy')

In [12]:
med = []
var = []
sky = []

In [13]:
from scipy.stats import skew

In [14]:
save_path = '/home/joao/Documentos'

In [15]:
for i in range(0,len(X[0])):
    
    templist = []
    #templist2 = []
    count = 0
    for j in range(0,len(X)):
        
        if X[j][i] > 0:
            
            templist.append(X[j][i])
            count += 1
    
    if count > 5:
        
        variance = np.var(templist)
        med.append(np.mean(templist))
        var.append(variance)
        sky.append(skew(templist)*(np.sqrt(variance))**3)
        
    else:
        
        med.append(0)
        var.append(0)
        sky.append(0)

In [16]:
Z = interp.griddata((xx,yy),med,(B1,B2),method='cubic')

In [17]:
teste = Z.transpose()

In [18]:
exp_x = []
exp_y = []
exp_z = []

for i in range(0,len(x)):
    
    for j in range(0,len(y)):
        
        exp_x.append(x[i])
        exp_y.append(y[j])
        exp_z.append(teste[i][j])
        

In [19]:
table = []
for l in range(0,len(exp_x)):
    table.append((exp_x[l],exp_y[l],exp_z[l]))


completeName = os.path.join(save_path, "med.dat")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()

In [20]:
Z = interp.griddata((xx,yy),var,(B1,B2),method='cubic')

In [21]:
teste = Z.transpose()

In [22]:
exp_x = []
exp_y = []
exp_z = []

for i in range(0,len(x)):
    
    for j in range(0,len(y)):
        
        exp_x.append(x[i])
        exp_y.append(y[j])
        exp_z.append(teste[i][j])
        

In [23]:
table = []
for l in range(0,len(exp_x)):
    table.append((exp_x[l],exp_y[l],exp_z[l]))


completeName = os.path.join(save_path, "var.dat")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()


In [24]:
Z = interp.griddata((xx,yy),sky,(B1,B2),method='cubic')

In [25]:
teste = Z.transpose()

In [26]:
exp_x = []
exp_y = []
exp_z = []

for i in range(0,len(x)):
    
    for j in range(0,len(y)):
        
        exp_x.append(x[i])
        exp_y.append(y[j])
        exp_z.append(teste[i][j])
        

In [27]:
table = []
for l in range(0,len(exp_x)):
    table.append((exp_x[l],exp_y[l],exp_z[l]))


completeName = os.path.join(save_path, "sky.dat")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()
